## EXPLORATION_04

### 인공지능 작사가 만들기

- keyword: NLP(Natural Language Processing)/ 순환신경망(RNN) 

#### lubric
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가? 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성된다
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가? 특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었다
3. 텍스트 생성모델이 안정적으로 학습되었는가? 텍스트 생성모델의 validation loss가 2.2 이하로 낮다

#### to-do list
1. 데이터 준비>데이터 정제
2. 평가 데이터셋 분리> 인공지능 만들기> loss 시각화
3. 모델 평가


#### trial and error

### 데이터 준비

In [1]:
import os, re 
import glob

import numpy as np
import tensorflow as tf

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof', 'You saw her bathing on the roof', 'Her beauty and the moonlight overthrew her', 'She tied you', 'To a kitchen chair', 'She broke your throne, and she cut your hair', 'And from your lips she drew the Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah You say I took the name in vain', "I don't even know the name"]


In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뛰기
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뛰기

    if idx > 9: break   # 일단 문장 10개만 확인
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


#### 데이터 정제

In [4]:
#입력된 문장을
#1. 소문자로 바꾸고, 양쪽 공백을 삭제
#2. 특수문자 양쪽에 공백을 넣고
#3. 여러개의 공백은 하나의 공백으로 바꾸기
#4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸기
#5. 다시 양쪽 공백 삭제
#6. 문장 시작에는 <start>, 끝에는 <end>를 추가
#7. 노래 구조 삭제 ex. [Outro]
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    sentence = re.sub("\[.*\]*", " ", sentence) # 7
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [5]:
#정제된 문장을 모으기
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛰기
    if len(sentence) == 0: continue # 길이가 0이거나 
    if len(sentence.split()) > 15 and len(sentence.split()) < 3 : continue # 토큰이 3보다 작거나 15개가 넘는 문장은 건너뛰기
        
 
    # 정제를 하고 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 확인
print(len(corpus))
corpus[:10]

175986


['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

#### Tokenization

In [6]:
import tensorflow as tf

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 단어장의 크기는 12,000 이상 으로 설정
        filters=' ',
        oov_token="<unk>"
    )
    # corpus로 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게함
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  
    print(tensor,tokenizer)
    return tensor, tokenizer

In [7]:
tensor, tokenizer = tokenize(corpus)

tensor.shape

[[   2   50    5 ...    0    0    0]
 [   2   17 2643 ...    0    0    0]
 [   2   35    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1014    3]
 [  37   15 9061 ...  878  644    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0720747340>


(175986, 15)

In [8]:
# 단어사전 확인

len(tokenizer.index_word)
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break #단어장의 10번째 단어까지 출력

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


### 평가 데이터셋 분리

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   64   57    9  970 6048    3    0    0    0]
[  50    5   91  297   64   57    9  970 6048    3    0    0    0    0]


In [10]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 2)
enc_val_train, enc_val_val, dec_val_train, dec_val_val = train_test_split(enc_train, dec_train, test_size = 0.125 , random_state = 2) 
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Val:", enc_val.shape)
print("Target Val:", dec_val.shape)

Source Train: (140788, 14)
Target Train: (140788, 14)
Source Val: (35198, 14)
Target Val: (35198, 14)


In [11]:
BUFFER_SIZE = len(enc_val_train) #텐서의 1차원, 전체 문장의 개수
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개 + 0:<pad>를 포함
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만들기
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

#### 모델의 정의

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

#### [Model]

In [13]:
embedding_size = 256 #단어 하나의 특징 수
hidden_size = 2048 #퍼셉트론의 개수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [14]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣기
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.62190452e-04, -2.20601542e-05,  4.90402635e-05, ...,
         -1.82441028e-04, -2.68238800e-04,  3.12287535e-04],
        [-3.48532558e-05, -2.89317017e-04,  7.54615685e-05, ...,
         -4.18709969e-04, -5.91588963e-04,  4.60059557e-04],
        [-3.15328616e-05, -2.75376806e-04,  1.44471283e-04, ...,
         -2.53642036e-04, -7.68889731e-04,  5.78273030e-04],
        ...,
        [ 2.96007143e-04, -1.75639179e-05,  1.02908125e-04, ...,
          3.12125077e-04, -1.14619143e-05,  5.82600420e-04],
        [ 5.64836781e-04, -2.16517874e-04, -2.95500882e-04, ...,
          1.92838546e-04, -1.25597198e-05,  7.79286551e-04],
        [ 7.88917649e-04, -4.76359040e-04, -6.57383760e-04, ...,
          1.26471918e-04, -1.15979601e-05,  9.70039051e-04]],

       [[ 1.62190452e-04, -2.20601542e-05,  4.90402635e-05, ...,
         -1.82441028e-04, -2.68238800e-04,  3.12287535e-04],
        [ 1.77651644e-04, -4.82845826e-05, -4

In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [18]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val),epochs=10, batch_size=512)

Epoch 1/10
275/275 [==============================] - 258s 930ms/step - loss: 2.8778 - val_loss: 2.9051
Epoch 2/10
275/275 [==============================] - 261s 950ms/step - loss: 2.6902 - val_loss: 2.8049
Epoch 3/10
275/275 [==============================] - 262s 954ms/step - loss: 2.5114 - val_loss: 2.7169
Epoch 4/10
275/275 [==============================] - 262s 955ms/step - loss: 2.1613 - val_loss: 2.5841
Epoch 6/10
275/275 [==============================] - 263s 957ms/step - loss: 1.9977 - val_loss: 2.5340
Epoch 7/10
275/275 [==============================] - 267s 973ms/step - loss: 1.8419 - val_loss: 2.4975
Epoch 8/10
275/275 [==============================] - 268s 975ms/step - loss: 1.6954 - val_loss: 2.4699
Epoch 9/10
275/275 [==============================] - 264s 961ms/step - loss: 1.5577 - val_loss: 2.4452
Epoch 10/10
275/275 [==============================] - 262s 955ms/step - loss: 1.4318 - val_loss: 2.4310


In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        #1. 입력받은 문장의 텐서를 입력
        predict = model(test_tensor) 
        #2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        #3. 2에서 예측된 word index를 문장 뒤에 붙입니다
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        #4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [23]:
print(generate_text(model, tokenizer, init_sentence="<start> i love"))
print(generate_text(model, tokenizer, init_sentence="<start> you re"))
print(generate_text(model, tokenizer, init_sentence="<start> a letter"))
print(generate_text(model, tokenizer, init_sentence="<start> you make"))

<start> i love you , i love you <end> 
<start> you re the only one who ever drove me crazy <end> 
<start> a letter full of coke rental car from <unk> <end> 
<start> you make me wanna get all their rules <end> 


#### trial and error

 NLP라는 분야도 처음 접해보고 RNN이란 개념도 처음 공부해봤다. 이번 프로젝트는 사실 시작하기도 전부터 조금 잘해보고 싶었던 분야다. '인공지능 작사가 만들기' 자체가 흥미로웠고 그래서 더 욕심이 나 마지막까지 이러고 있다... 
하지만 생각보다 아직 내 실력이 부족해서 흥미로운 결과를 얻진 못했다. 따로 한글 가사 데이터셋을 구성해서 모델링을 해봤는데 생각보다 결과가 좋지 않았고 하이퍼 파라미터를 조정하는 부분도 미숙했기 때문이다. 
그래도 이제 루브릭 평가 지표를 2개를 만족시키는 학습은 3-4시간만 투자하면 얻어낼 수 있다는 사실에 약 한 달이 채 되지 않는 시간동안 많이 경험을 해본 것 같다. 
- 이번 프로젝트에서 토큰화를 처음 진행해 보았는데 다음에 기회가 된다면 데이터 전처리, 하이퍼 파라미터 조정에 대해 더 깊은 공부를 해보고 싶다.
- num_words, batch size, embedding size, hidden size 등 수치 변화를 통해 validation loss를 맞추려고 여러차례 반복해봤는데 시간이 너무 오래걸려 결국 마지막 날 마지막 시간까지 조정하고 기다렸다. 아래에 삽입한 이미지는 근접했던 loss 값.
- 마지막에 Total params: 257,794,017까지 나온건 잊지 못할 듯